# 04: Effective Go

These are notes from https://go.dev/doc/effective_go

## Objectives 

TBD

## Introduction

This guide will let you understand Go's properties and idioms. Additionally, it will be explained Go's conventions relative to naming, formatting, program construction, etc.

| NOTE: |
| :---- |
| This [guide](https://go.dev/doc/effective_go) was written in 2009, and hasn't been updated much since. While it continues to be useful, a few of the examples will feel outdated. |

## Formatting

Go includes the *gofmt* program, available as `go fmt` that reads a Go program and emits the source in a standard style of indentation and vertical alignment, retaining and if necessary reformatting comments.

For example:

The *untidied*:
```go
type T struct {
	name string // name of the object
	value int // its value
}
```

Will be transformed by *gofmt* into:

```go
type T struct {
	name  string // name of the object
	value int    // its value
}
```

| NOTE: |
| :---- |
| You must configure VSCode with `gofmt` in the settings. |

Additionally:
+ Tabs are used for indentation, and *gofmt* emits them by default. Spaces should be used only if you must.

    Because of this, it's not that important whether you set your tab size to 2, 4 or 8. I use two because it makes the code more compact horizontally.

+ There is no length limit in Go. If a line feels too long, wrap it and indent with an extra tab.

+ Go needs fewer parentheses than C and Java.

## Commentary

Go supports C-style `/* */` block comments and C++-style `//` line comments, with the latter being the norm.

## Names

Names in Go are extremely important, as they even have semantic effects (e.g., the visibility of a name outside of a package is determined by whether its first character is upper case).

### Package names

When a package name is imported, the package name becomes the accessor for the contents.

That is, after:

```go
import "bytes"
```

you can do `bytes.Buffer`.

By convention, packages are given lowercase, single-word names; there should be no need for underscore or mixedcaps. The package names should be short, concise, and evocative.

Another convention is that the package name is the base name of its source directory. For example, a package in `src/encoding/base64` will be imported as `import "encoding/base64"` and the package name will be `base64` and not `encoding_base64` or similar.

Additionally, think of the end-users: when exposing a function in your package use a concise name like `Reader` instead of `BufReader`, as the users will refer to it as `bufio.Reader`.

Similarly, when creating new instances of a particular type defined in a package, it is recommended to use something like `New()` instead of `NewRing()` if the package only exports a single type (as it should). This is because the end-users will code `ring.New()` to refer to it.

In Go, short names with a helpful doc comment is preferred over a long name:

```go
// discouraged
once.DoOrWaitUntilDone(setup)

// preferred
once.Do(setup) // blocks until setup is completed
```

### Getters

Go doesn't provide automatic support for getters and setters. 

This doesn't mean that you cannot use them, but it's not consider idiomatic, nor necessary to put `Get` into the getter's name.

In Go, it's preferred to use the name of the object (uppercase, exported) instead of `GetField()`, while for the setter it is OK to use `SetField()`

```go
// Person is a struct used in the sample
type Person struct {
	name string // name of the object
	age  int
}

// Name is the Person.name getter
func (p *Person) Name() string {
	return p.name
}

// Age is the Person.age getter
func (p *Person) Age() int {
	return p.age
}

// SetName is the Person.name setter
func (p *Person) SetName(name string) {
	p.name = name
}

// Age is the Person.name setter
func (p *Person) SetAge(age int) {
	p.age = age
}

func main() {
	var p Person

	p.SetName("Jason Isaacs")
	p.SetAge(55)

	fmt.Println(p)

	fmt.Printf("Hello to %v\n", p.Name())
}
```

### Interface names

By convention, one-method interfaces are named by the method name plus an *-er* suffix or similar:

```go
type Reader interface {
  Read(f *File, b []byte) (n int, err error)
}
```

There are a number of such names (`Reader`, `Writer`, `Formatter`, `CloseNotifier`...) and it's productive to honor them and the function names they capture.

There are also canonical signatures and meanings such as `String()`, `Error()`, `Close()`, `Flush()`, etc.

To avoid confusion, don't give your method such names unless it has the same signature and meaning.

Conversely, if your type implements a method with the same meaning as a method on a well-known type, give it the same name and signature (e.g., call your string converter method `String()` and not `ToString()`).

```go
// Person is a struct used in the sample
type Person struct {
	name string // name of the object
	age  int
}

func (p *Person) String() string {
	return fmt.Sprintf("%q (%v)", p.name, p.age)
}

func main() {
	var p *Person = &Person{"Jason Isaacs", 56}

	fmt.Println(p)
}
```

### MixedCaps

The convention in Go is to use MixedCaps or mixedCaps rather than underscores to write multiWord names.

## Semicolons

Go's formal grammar uses semicolons to terminate statements, but unlike C, those semicolons do not appear in the source &mdash; the lexer includes them automatically as it scans the source code.

However, the rules the lexer uses imposes certain conventions, such as having to put the opening brace of a control structure in the same line and not on the next line:

```go
// Correct
if i < f() {
  g()
}

// WRONG!
if i < f()
{
  g()
}
```

| NOTE: |
| :---- |
| The compiler and IDE will warn you of that even with an "unexpected newline" error. |

## Control structures

In Go:
+ There is no `do` or `while` loop, but there is a generalized form of `for` that fits the bill.
+ `switch` is far more flexible than in other programming languages.
+ `if` and `switch` accept an optional initialization statement (like the one you have in `for` loops).
+ `break` and `continue` support an optional label to identify what to break or continue.
+ There is a type `switch`
+ There is a `select` multiway communication multiplexer.


### If

In Go a simple `if` looks like:

```go
if x > 0 {
  return y
}
```

That is, you need to use the braces no matten how simple the if body is.

"If" statement in Go supports an initialization statement:

```go
if err := file.Chmod(0664); err != nil {
  log.Print(err)
  return err
}
```

In Go, it is also considered idiomatic to code if statements that doesn't flow into the next statement &mdash; that is, the body ends in break, continue, goto, or return:

```go
f, err := os.Open(name)
if err != nil {
  return err
}

codeUsing(f)
```

The following code shows another example in which you will find no "else" statements. In it, flow control runs does the page, eliminating error cases as they arise:

```go
f, err := os.Open(name)
if err != nil {
  return err  // Couldn't open the file
}

d, err := f.Stat()  // redeclaration of err? (see below)
if err != nil {
  f.Close()
  return err  // Couldn't stat the file
}

codeUsing(f, d)
```

### Redeclaration and assignment

In the last example, we see that we do:

```go
f, err := os.Open(name)
```

and a few lines later:

```go
d, err := f.Stat()
```

The duplication of `err` is legal: `err` is declared by the first statement, and only re-assigned in the second (that is, the call to `f.Stat()` does not declares a new variable, it just gives it a new value).

In a `:=` declaration, a variable `v` may appear even if it has already been declared, provided that:

+ this declaration is in the same scope as the existing declaration of `v`. If `v` is already declared in an outer scope, the declaration will create a new variable,

+ the corresponding value in the initialization is assignable to `v`, and

+ there is at least one other variable that is created by the declaration.

### For

### Switch

### Type Switch

## Functions

### Multiple Return Values

### Named Result Parameters

### Defer

## Data

### Allocation with new

### Constructors and composite literals

### Allocation with make

### Arrays

### Slices

### Two-dimensional slices

### Maps

### Printing

### Append

## Initialization

### Constants

### Variables

### The init function

## Methods

### Pointers vs. Values

## Interfaces and other types

### Interfaces

### Conversions

### Interface conversions and type assertions

### Generality

### Interfaces and methods

## The blank identifier

### The blank identifier in multiple assignment

### Unused imports and variables

### Import for side effect

### Interface checks

## Embedding

## Concurrency

### Share by communicating

### Goroutines

### Channels

### Channels of channels

### Parallelization

### A leaky buffer

## Errors

### Panic

### Recover

## A web server